In [ ]:
import calendar_utils
import json
from dotenv import load_dotenv
import os
import re
import datetime
import openai
from openai import OpenAI

load_dotenv()

In [ ]:
class CalendarChatGPT:
  def __init__(self, openai_api_key, model ="gpt-4-1106-preview", json_output=False, max_tokens=100):
    
    openai.api_key = openai_api_key
    self.max_tokens = max_tokens
    self.messages = []
    
    self.json_output = json_output
    
    self.client = OpenAI()
    self.model = model
    self.service = calendar_utils.get_calendar_service()

  def call(self):
    if self.json_output:
      response = self.client.chat.completions.create(
        model=self.model,
        response_format={ "type": "json_object" }, 
        messages=self.messages,
        max_tokens=self.max_tokens
      )
    else:
      response = self.client.chat.completions.create(
        model=self.model,
        messages=self.messages,
        max_tokens=self.max_tokens
      )
    return response

  def prompt(self, text):
    
    # Call ChatGPT for intent classification
    user_prompt = (
        f"{text}\n\nDo not respond yet. Classify user intention as one of the following."
        "(1) might be relevant to your response, separated by commas."
        "If none, just say nothing."
    )
    
    print(">>===========================================")
    print("[Prompt for NER]")
    print(user_prompt + "\n")

    # Update the messages
    self.messages.append({
        "role": "user",
        "content": user_prompt,
    })

    # Call ChatGPT
    response = self.call()

    # Parse the ChatGPT response to obtain a list of entities
    message_content = response["choices"][0]["message"]["content"]

    
    
    # Update the messages so far
    self.messages.append({
        "role": "user",
        "content": text,
    })

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response["choices"][0]["message"]
    self.messages.append(message)

    return message["content"]


In [ ]:
# Wikipedia-grounded ChatGPT class
import openai
import wikipediaapi

class WikiChatGPT:
  def __init__(self, openai_api_key, max_tokens=100):
    openai.api_key = openai_api_key
    self.max_tokens = max_tokens
    self.messages = []

    # Add wikipedia module
    self.wikipedia = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

  def call(self):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=self.messages,
      max_tokens=self.max_tokens,
    )
    return response

  def prompt(self, text):
    # Call ChatGPT for named entity recognition
    user_prompt = (
        f"{text}\n\nDo not respond yet. List at most two named entities that "
        "might be relevant to your response, separated by commas."
        "If none, just say nothing."
    )

    print(">>===========================================")
    print("[Prompt for NER]")
    print(user_prompt + "\n")

    # Update the messages
    self.messages.append({
        "role": "user",
        "content": user_prompt,
    })

    # Call ChatGPT
    response = self.call()

    # Parse the ChatGPT response to obtain a list of entities
    message_content = response["choices"][0]["message"]["content"]

    print("[ChatGPT response for NER]")
    print(message_content + "\n")
    entities = [
        entity.strip() for entity in message_content.split(",")
        if entity.strip().lower() not in ["", "nothing"]
    ]

    # Retrieve Wikipedia pages
    entity_descriptions = []
    for entity in entities:
        page = self.wikipedia.page(entity)
        entity_descriptions.append(f"{entity}: {page.summary}")
    entity_descriptions_str = "\n".join(entity_descriptions)

    print("[Wikipedia retrieval]")
    if entity_descriptions_str:
        print(entity_descriptions_str)
    print()

    # Call ChatGPT to get a response to the original prompt
    user_prompt = f"{text}\n\n"
    if entity_descriptions_str:
        user_prompt += (
            "Incorporate the following information in your response if necessary:\n"
            f"{entity_descriptions_str}"
        )
    print("[Revised user prompt]")
    print(user_prompt)
    print("===========================================<<")

    # Update the messages
    self.messages[-1]["content"] = user_prompt

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response["choices"][0]["message"]
    self.messages.append(message)

    return message["content"]